In [29]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [77]:
image = cv2.imread("output/Enhanced1.jpg",0)
w,r=image.shape
image=cv2.resize(image,(int(r/50),int(w/50)))
cv2.imwrite('output/resize.jpg',image)
image=cv2.resize(image,(r,w))
cv2.imwrite('output/resize.jpg',image)

True

In [71]:
Z = image.reshape((-1,3))
Z = np.float32(Z)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 4
ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
center = np.uint8(center)
res = center[label.flatten()]
res2 = res.reshape((image.shape))
cv2.imwrite('output/K-Means.jpg',res2)

True

In [32]:
res2 = cv2.cvtColor(res2,cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(res2,(5,5),0)
ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
cv2.imwrite("output/thresholding.jpg",th3)

True

In [78]:

ret3,th3 = cv2.threshold(image,80,255,cv2.THRESH_BINARY)
cv2.imwrite("output/patching.jpg",th3)
sizeX = th3.shape[1]
sizeY = th3.shape[0]
nRows = 150
mCols = 150
x_increment = int(sizeY/nRows)
y_increment = int(sizeX/mCols)

In [73]:
for i in range(0,nRows):
    for j in range(0, mCols):
        roi = th3[int(i*x_increment):int(i*x_increment + x_increment) ,int(j*y_increment):int(j*y_increment + y_increment)]
        count = cv2.countNonZero(roi)
        if 100*count/roi.size >= 60:
            th3[int(i*x_increment):int(i*x_increment + x_increment) ,int(j*y_increment):int(j*y_increment + y_increment)] = 255
        else:
            th3[int(i*x_increment):int(i*x_increment + x_increment) ,int(j*y_increment):int(j*y_increment + y_increment)] = 0
cv2.imwrite("output/patching.jpg",th3)

True

In [60]:
pixeled = th3
rows =  pixeled.shape[0]
cols = pixeled.shape[1]

In [61]:
points = list()
lower_jaw = pixeled.copy()
#jaw lowest point
for i in range(rows-15,0,-1):
    for j in range(int(cols*3/4),int(cols/2),-1):
        if pixeled[i][j] == 255:
            points.append((i,j))
            flag = 1
            break
        else:
            flag = 0
    if flag == 1:
        break
        
#midpoint
mid1 = [points[0][0],int(points[0][1] + ((pixeled.shape[1] - points[0][1]) /2))]
mid2 = [points[0][0],int(((points[0][1]) /2))] 
lower_jaw[mid1[0],mid1[1]] = 150
lower_jaw[mid2[0],mid2[1]] = 150
r = points[0][0]
for i in range(r-1,1,-1):
    midr = mid1[1]
    midl = mid2[1]
    lower_jaw[i][midr] = 150
    lower_jaw[i][midl] = 150
    for j in range(midr,0,-1):
        if pixeled[i][j] == 255:
            lower_jaw[i][j] = 200
            points.append((i,j))
            break
    for j in range(midl,pixeled.shape[1]):
        if pixeled[i][j] == 255:
            lower_jaw[i][j] = 200
            points.append((i,j))
            break
        
cv2.imwrite('output/lower_jaw.jpg',lower_jaw)           

True

In [62]:
black = np.zeros((pixeled.shape[0],pixeled.shape[1],3))
for i,j in points:
    black[i][j] = [255,0,0]
cv2.imwrite('output/points.jpg',black)

True

equalized1 = cv2.equalizeHist(image)
image = equalized1

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
equalized = clahe.apply(image)
filtered = cv2.medianBlur(equalized,3)
#ret2,th2 = cv2.threshold(filtered,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
image = filtered
G = image.copy()
for i in range(5,0,-1):
    G = cv2.pyrDown(G)
kernel = np.ones((5,5),np.uint8)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
#for i in range(5,0,-1):
   # G = cv2.pyrUp(G)
image = opening
cv2.imwrite("Results/enhanced_image.jpg",image)

ret, thresh = cv2.threshold(image,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
image = cv2.reshape(thresh,image1.shape)
#image = cv2.resize(thresh,(width,height))

image = image.reshape(-1,3)
Z = np.float32(image)

# Define criteria = ( type, max_iter = 10 , epsilon = 1.0 )
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)

# Set flags (Just to avoid line break in the code)
flags = cv2.KMEANS_RANDOM_CENTERS

# Apply KMeans
compactness,labels,centers = cv2.kmeans(Z,2,None,criteria,10,flags)
res = centers[labels.flatten()]
res2 = res.reshape(image1.shape)
cv2.imwrite("kmeans.jpg", res2)

contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
new_image = cv2.drawContours(image, contours, -1, (100,255,50), 3)
cv2.imwrite('counting.jpg',new_image)

image = img_as_float(image)
# Initial level set
init_ls = checkerboard_level_set(image.shape, 6)
# List with intermediate results for plotting the evolution
evolution = []
callback = store_evolution_in(evolution)
ls = morphological_chan_vese(image, 35, init_level_set=init_ls, smoothing=3,
                             iter_callback=callback)
#ontours = scipy.sparse.csr_matrix.toarray(ls)

fig, axes = plt.subplots(2, 2, figsize=(8, 8))
ax = axes.flatten()

ax[0].imshow(image, cmap="gray")
ax[0].set_axis_off()
image1 = ax[0].contour(ls, [0.5], colors='r')
ax[0].set_title("Morphological ACWE segmentation", fontsize=12)

ax[1].imshow(ls, cmap="gray")
ax[1].set_axis_off()
title = "Morphological ACWE evolution"
ax[1].set_title(title, fontsize=12)



#print(type(contours))
#print(len(ls[20]))
#print(evolution[-1])
#brect = cv2.boundingRect(contours)

#cv2.imwrite("morph1.jpg" , ls)

#jaw = cv2.bitwise_and(image,image,mask = ls)
#cv2.imwrite('jaw2.jpg' , jaw)